# Pandas - Working with JSON

## Writing JSON

__*to_json()*__

A __Series__ or __DataFrame__ can be converted to a valid __JSON__ string. Use __*to_json*__ with optional parameters

In [2]:
import pandas as pd
import numpy as np
import io
from cStringIO import StringIO
import csv

In [3]:
df = pd.DataFrame(np.random.randint(0,10,(5,2)), columns=list('AB'))
df

,A,B
0,3,1
1,0,1
2,2,7
3,1,4
4,4,6


In [4]:
json = df.to_json()
json

'{"A":{"0":3,"1":0,"2":2,"3":1,"4":4},"B":{"0":1,"1":1,"2":7,"3":4,"4":6}}'

## Orient Options

__*orient*__

There are a number of different options for the format of the resulting JSON file / string.

In [5]:
df = pd.DataFrame(dict(A=range(1,4),
                       B=range(4,7),
                       C=range(8,11)),
                  index=list('xyz'))
df

,A,B,C
x,1,4,8
y,2,5,9
z,3,6,10


In [6]:
S = pd.Series(dict(x=15, y=16, z=17), name='D')
S

x    15
y    16
z    17
Name: D, dtype: int64

### Column oriented

serializes the data as nested JSON objects with column labels acting as the primary index

In [7]:
df.to_json(orient='columns')

'{"A":{"x":1,"y":2,"z":3},"B":{"x":4,"y":5,"z":6},"C":{"x":8,"y":9,"z":10}}'

### Index orinted

similar to column oriented but the index labels are now primary

In [8]:
df.to_json(orient='index')

'{"x":{"A":1,"B":4,"C":8},"y":{"A":2,"B":5,"C":9},"z":{"A":3,"B":6,"C":10}}'

In [9]:
S.to_json(orient='index')

'{"x":15,"y":16,"z":17}'

### Record oriented

serializes the data to a JSON array of column -> value records, index labels are not included. This is useful for passing DataFrame data to plotting libraries, for example the JavaScript library d3.js

In [10]:
df.to_json(orient='records')

'[{"A":1,"B":4,"C":8},{"A":2,"B":5,"C":9},{"A":3,"B":6,"C":10}]'

In [11]:
S.to_json(orient='records')

'[15,16,17]'

### Value oriented

s a bare-bones option which serializes to nested JSON arrays of values only, column and index labels are not included

In [12]:
df.to_json(orient='values')

'[[1,4,8],[2,5,9],[3,6,10]]'

### Split oriented

serializes to a JSON object containing separate entries for values, index and columns. Name is also included for Series

In [13]:
df.to_json(orient='split')

'{"columns":["A","B","C"],"index":["x","y","z"],"data":[[1,4,8],[2,5,9],[3,6,10]]}'

In [14]:
S.to_json(orient='split')

'{"name":"D","index":["x","y","z"],"data":[15,16,17]}'

### Date handling

In [15]:
df = pd.DataFrame(np.random.randint(0,10,(5,2)), 
                  columns=list('AB'))
df['date'] = pd.date_range('20170401', periods=5, freq='M')
df

,A,B,date
0,7,1,2017-04-30
1,7,3,2017-05-31
2,3,4,2017-06-30
3,8,8,2017-07-31
4,6,8,2017-08-31


### Wirting in ISO format

__*date_format*__

__*date_unit*__

In [16]:
df.sort_index(1, ascending=False)

,date,B,A
0,2017-04-30,1,7
1,2017-05-31,3,7
2,2017-06-30,4,3
3,2017-07-31,8,8
4,2017-08-31,8,6


In [17]:
df.to_json(date_format='iso')

'{"A":{"0":7,"1":7,"2":3,"3":8,"4":6},"B":{"0":1,"1":3,"2":4,"3":8,"4":8},"date":{"0":"2017-04-30T00:00:00.000Z","1":"2017-05-31T00:00:00.000Z","2":"2017-06-30T00:00:00.000Z","3":"2017-07-31T00:00:00.000Z","4":"2017-08-31T00:00:00.000Z"}}'

### Writing in ISO date format, with microseconds

In [18]:
df.to_json(date_format='iso', date_unit='us')

'{"A":{"0":7,"1":7,"2":3,"3":8,"4":6},"B":{"0":1,"1":3,"2":4,"3":8,"4":8},"date":{"0":"2017-04-30T00:00:00.000000Z","1":"2017-05-31T00:00:00.000000Z","2":"2017-06-30T00:00:00.000000Z","3":"2017-07-31T00:00:00.000000Z","4":"2017-08-31T00:00:00.000000Z"}}'

### Epoch timestamps in seconds

In [19]:
df.to_json(date_format='epoch', date_unit='s')

'{"A":{"0":7,"1":7,"2":3,"3":8,"4":6},"B":{"0":1,"1":3,"2":4,"3":8,"4":8},"date":{"0":1493510400,"1":1496188800,"2":1498780800,"3":1501459200,"4":1504137600}}'

### Writing to file, with a date index and a date column

In [20]:
df2 = df.copy()
df2['date'] = pd.Timestamp('20170101')
df2['ints'] = list(range(5))
df2['bools'] = True
df2.index = pd.date_range('20170401', periods=5, freq='M')
df2

,A,B,date,ints,bools
2017-04-30,7,1,2017-01-01,0,True
2017-05-31,7,3,2017-01-01,1,True
2017-06-30,3,4,2017-01-01,2,True
2017-07-31,8,8,2017-01-01,3,True
2017-08-31,6,8,2017-01-01,4,True


In [21]:
df2.to_json('test.json')
open('test.json').read()

'{"A":{"1493510400000":7,"1496188800000":7,"1498780800000":3,"1501459200000":8,"1504137600000":6},"B":{"1493510400000":1,"1496188800000":3,"1498780800000":4,"1501459200000":8,"1504137600000":8},"date":{"1493510400000":1483228800000,"1496188800000":1483228800000,"1498780800000":1483228800000,"1501459200000":1483228800000,"1504137600000":1483228800000},"ints":{"1493510400000":0,"1496188800000":1,"1498780800000":2,"1501459200000":3,"1504137600000":4},"bools":{"1493510400000":true,"1496188800000":true,"1498780800000":true,"1501459200000":true,"1504137600000":true}}'

## Reading JSON

In [22]:
df = pd.DataFrame(np.random.randint(0,10,(5,2)), 
                  columns=list('AB'))
df['date'] = pd.date_range('20170401', periods=5, freq='M')
df = df.sort_index(1, ascending=False)
json = df.to_json()
json

'{"date":{"0":1493510400000,"1":1496188800000,"2":1498780800000,"3":1501459200000,"4":1504137600000},"B":{"0":3,"1":1,"2":6,"3":5,"4":5},"A":{"0":2,"1":4,"2":7,"3":3,"4":3}}'

In [23]:
dfj = pd.read_json(json)
dfj

,A,B,date
0,2,3,2017-04-30
1,4,1,2017-05-31
2,7,6,2017-06-30
3,3,5,2017-07-31
4,3,5,2017-08-31


In [24]:
pd.read_json('test.json')

,A,B,bools,date,ints
2017-04-30,7,1,True,2017-01-01,0
2017-05-31,7,3,True,2017-01-01,1
2017-06-30,3,4,True,2017-01-01,2
2017-07-31,8,8,True,2017-01-01,3
2017-08-31,6,8,True,2017-01-01,4


In [25]:
pd.read_json('test.json', dtype=object).dtypes

A        object
B        object
bools    object
date     object
ints     object
dtype: object

In [26]:
pd.read_json('test.json', dtype={'A':float, 'bools':int}).dtypes

A               float64
B                 int64
bools             int32
date     datetime64[ns]
ints              int64
dtype: object

In [27]:
si = pd.DataFrame(np.zeros((4, 4)),
                 columns=list(range(4)),
                 index=[str(i) for i in range(4)])
si

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


In [28]:
si.index

Index([u'0', u'1', u'2', u'3'], dtype='object')

In [29]:
si.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [30]:
json = si.to_json()

In [31]:
sj = pd.read_json(json, convert_axes=False)
sj

,0,1,2,3
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0


In [32]:
sj.index

Index([u'0', u'1', u'2', u'3'], dtype='object')

In [33]:
sj.columns

Index([u'0', u'1', u'2', u'3'], dtype='object')

### Dates writteb in nanoseconds need to be read back in nanoseconds

In [34]:
df = pd.DataFrame(np.random.randint(0,10,(5,2)), 
                  columns=list('AB'))
df['date'] = pd.date_range('20170401', periods=5, freq='M')
df = df.sort_index(1, ascending=False)
json = df.to_json(date_unit='ns')
json

'{"date":{"0":1493510400000000000,"1":1496188800000000000,"2":1498780800000000000,"3":1501459200000000000,"4":1504137600000000000},"B":{"0":1,"1":0,"2":7,"3":5,"4":1},"A":{"0":8,"1":6,"2":2,"3":8,"4":0}}'

In [35]:
dfj = pd.read_json(json, date_unit='us')
dfj

,A,B,date
0,8,1,1493510400000000000
1,6,0,1496188800000000000
2,2,7,1498780800000000000
3,8,5,1501459200000000000
4,0,1,1504137600000000000


In [36]:
dfj = pd.read_json(json, date_unit='ns')
dfj

,A,B,date
0,8,1,2017-04-30
1,6,0,2017-05-31
2,2,7,2017-06-30
3,8,5,2017-07-31
4,0,1,2017-08-31


### Numpy Parameters

If __*numpy=True*__ is passed to read_json an attempt will be made to sniff an appropriate dtype during deserialization and to subsequently decode directly to numpy arrays, bypassing the need for intermediate Python objects.

This can provide speedups if you are deserialising a large amount of numeric data

In [37]:
rndfs = np.random.uniform(-100, 100, 10000)
rndfs.shape = (1000, 10)
dfrnd = pd.DataFrame(rndfs, columns=list('ABCDEFGHIJ'))
jsonrnd = dfrnd.to_json()

In [38]:
%timeit pd.read_json(jsonrnd)

100 loops, best of 3: 9.83 ms per loop


In [39]:
%timeit pd.read_json(jsonrnd, numpy=True)

100 loops, best of 3: 6.13 ms per loop


### Normalization

pandas provides a utility function to take a dict or list of dicts and normalize this semi-structured data into a flat table.

In [40]:
from pandas.io.json import json_normalize

In [41]:
data = [{'state': 'Florida',
         'shortname': 'FL',
         'info': {
             'governor': 'Rick Scott'
             },
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {
             'governor': 'John Kasich'
             },
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]
data

[{'counties': [{'name': 'Dade', 'population': 12345},
   {'name': 'Broward', 'population': 40000},
   {'name': 'Palm Beach', 'population': 60000}],
  'info': {'governor': 'Rick Scott'},
  'shortname': 'FL',
  'state': 'Florida'},
 {'counties': [{'name': 'Summit', 'population': 1234},
   {'name': 'Cuyahoga', 'population': 1337}],
  'info': {'governor': 'John Kasich'},
  'shortname': 'OH',
  'state': 'Ohio'}]

In [42]:
json_normalize(data, 
               'counties', 
               ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


### Line delimited json

pandas is able to read and write line-delimited json files that are common in data processing pipelines using Hadoop or Spark.

In [43]:
jsonl = '''
        {"a":1,"b":2}
        {"a":3,"b":4}
    '''
jsonl

'\n        {"a":1,"b":2}\n        {"a":3,"b":4}\n    '

In [44]:
df = pd.read_json(jsonl, lines=True)
df

,a,b
0,1,2
1,3,4


In [45]:
df.to_json(orient='records', lines=True)

u'{"a":1,"b":2}\n{"a":3,"b":4}'